# Analysis of the influences of institutions on adoption of clean technology

## Data processing
Datasets used: 

From Passport by Euromonitor:
- Primary Energy Supply of Solar, Wind and Other Energy: Euromonitor International from International Energy Association (IEA)
- Primary Energy Supply: Euromonitor International from International Energy Association (IEA)
- Primary Energy Supply of Coal, Peat and Oil Shale: Euromonitor International from International Energy Association (IEA)
- Primary Energy Supply of Geothermal Energy: Euromonitor International from International Energy Association (IEA)
- Primary Energy Supply of Biofuels and Waste: Euromonitor International from International Energy Association (IEA)
- Primary Energy Supply of Hydro Energy: Euromonitor International from International Energy Association (IEA)
- Primary Energy Supply of Nuclear Energy: Euromonitor International from International Energy Association (IEA)
- Primary Energy Supply of Crude Oil and Natural Gas Liquids (NGL): Euromonitor International from International Energy Association (IEA)
- Property Rights Ranking: Euromonitor International from The Heritage Foundation
- Natural Resources Rents: Euromonitor International from World Bank

Other datasets
- World Income Inequality Database (WIID) Companion dataset (wiidcountry and/or wiidglobal)
- V-Dem [Country–Year/Country–Date] Dataset v11.1

### Import data

In [175]:
import pandas as pd
import numpy as np

In [176]:
solar_wind_other = pd.read_excel("./Datasets/Solar Wind Other.xls", header = 5)
all_energy = pd.read_excel("./Datasets/All energy.xls", header = 5, skipfooter = 6)
coal_peat_oil = pd.read_excel("./Datasets/Coal Peat Oil.xls", header = 5, skipfooter = 6)
geothermal = pd.read_excel("./Datasets/Geothermal.xls", header = 5)
biogas = pd.read_excel("./Datasets/Biogas.xls", header = 5)
hydro = pd.read_excel("./Datasets/Hydro.xls", header = 5)
nuclear = pd.read_excel("./Datasets/nuclear.xls", header = 5, skipfooter = 6)
oil_gas = pd.read_excel("./Datasets/Oil Gas.xls", header = 5)
property_rights = pd.read_excel("./Datasets/Property rights.xls", header = 5, skipfooter = 6)
rents = pd.read_excel("./Datasets/Resource rents.xls", header = 5)
wiid = pd.read_excel("./Datasets/WIID.xlsx")
democracy = pd.read_csv("./Datasets/democracy.csv")

### Process data

In [535]:
# Solar Wind Other
solar_processed = solar_wind_other.set_index("Geography")[["2015","2019"]].replace("-", 0)

# All energy
all_processed = all_energy.set_index("Geography")[["2015","2019"]].replace("-", 0)

# Coal Peat Oil
coal_processed = coal_peat_oil.set_index("Geography")[["2015","2019"]].replace("-", 0)

# Biogas
biogas_processed = biogas.set_index("Geography")[["2015","2019"]].replace("-", 0)

# Hydro
hydro_processed = hydro.set_index("Geography")[["2015","2019"]].replace("-", 0)

# Geothermal
geothermal_processed = geothermal.set_index("Geography")[["2015","2019"]].replace("-", 0)

# Nuclear
## Clean data
nuclear_processed = nuclear.set_index("Geography")[["2015","2019"]].replace("-", 0)
## Convert to million barrels of oil
# According to the US EIA, 1 Wh = 8.598e-8 tonnes of oil equivelent, and 1 KWh = 1000 Wh
kwh_to_tonnes_oil = 8.595E-8 * 1000 / 10E6
nuclear_processed = nuclear_processed.apply(lambda energy: energy * kwh_to_tonnes_oil)

# Oil Gas
oil_processed = oil_gas.set_index("Geography")[["2015","2019"]].replace("-", 0)

# Property Rights
property_processed = property_rights.set_index("Geography")[["2019"]]
property_processed = property_processed.rename(index={"USA": "United States"})
property_processed = property_processed.rename(columns = {"2019": "Property rights score"})

# Rents
rents_processed = rents.set_index("Geography")[["2019"]].replace("-", 0)
rents_processed = rents_processed.rename(index={"USA": "United States"})
rents_processed = rents_processed.rename(columns = {"2019": "Resource rents"})

# WIID
wiid_processed = wiid.set_index("country")[["year", "palma", "gdp", "quality", "oecd"]]
wiid_processed = wiid_processed.dropna()
wiid_processed = wiid_processed[wiid_processed["year"] > 2000]
groups = wiid_processed.groupby("country")
palma_processed_list = []

def filter_palma(group):
    df = group[1]   # Get dataframe
    most_recent_year = df.iloc[-1]["year"]   # Get the most recent year for which there's data, and ignore older data
    df_latest = df[df["year"] == most_recent_year]
    best_quality = "High" if df_latest["quality"].str.contains("High").any() else ("Average" if df_latest["quality"].str.contains("Average").any() else "Low")
    df_latest_best = df_latest[df_latest["quality"] == best_quality]   # Get best quality data, and ignore lower quality data
    mean_palma = df_latest_best["palma"].mean()  # If we still have multiple rows, average scores
    mean_gdp = df_latest_best["gdp"].mean()
    oecd = df_latest_best["oecd"].mode()
    return[group[0], mean_palma, mean_gdp, oecd]
    
for group in groups:
    palma_processed_list.append(filter_palma(group))

palma_processed = pd.DataFrame(palma_processed_list, columns = ["country", "palma", "gdp", "oecd"]).set_index("country")
gdp_processed = palma_processed["gdp"]
oecd_processed = palma_processed["oecd"].to_frame()
palma_processed = palma_processed["palma"].to_frame()
palma_processed = palma_processed.rename(columns = {"palma": "Palma"})

# Democracy
democracy_processed = democracy.set_index("country_name")[["year","v2x_polyarchy"]]
democracy_processed = democracy_processed.dropna()
democracy_processed = democracy_processed[democracy_processed["year"] > 2000] # Clear any data from before 2000
democracy_processed = democracy_processed.rename(index={"United States of America": "United States"})
# Keep only the data from the most recent year
democracy_processed = democracy_processed.reset_index().drop_duplicates(subset = ["country_name"], keep = 'last').set_index("country_name")
democracy_processed = democracy_processed.rename(columns = {"v2x_polyarchy": "Electoral democracy index"})
democracy_processed = democracy_processed["Electoral democracy index"]

### Synthesize data

In [564]:
all_renewables = pd.concat([solar_processed, biogas_processed, hydro_processed, geothermal_processed, nuclear_processed])
all_renewables = all_renewables.groupby("Geography")
all_renewables = all_renewables.sum()
all_renewables["renew_change"] = all_renewables["2019"] - all_renewables["2015"]
all_renewables = all_renewables.rename(index={"USA": "United States"})

all_nonrenewables = pd.concat([coal_processed, oil_processed])
all_nonrenewables = all_nonrenewables.groupby("Geography")
all_nonrenewables = all_nonrenewables.sum()
all_nonrenewables["nonrenew_change"] = all_nonrenewables["2019"] - all_nonrenewables["2015"]
all_nonrenewables = all_nonrenewables.rename(index={"USA": "United States"})

all_processed["energy_change"] = all_processed["2019"] - all_processed["2015"]

oecd_mapped = oecd_processed.applymap(lambda cell: 1 if cell[0] == "OECD" else 0)

country_lists = []
renewables_country_list = all_renewables.index.to_list()
country_lists.append(renewables_country_list)
nonrenewables_country_list = all_nonrenewables.index.to_list()
country_lists.append(nonrenewables_country_list)
energy_country_list = all_processed.index.to_list()
country_lists.append(energy_country_list)
property_country_list = property_processed.index.to_list()
country_lists.append(property_country_list)
rents_country_list = rents_processed.index.to_list()
country_lists.append(rents_country_list)
gdp_country_list = gdp_processed.index.to_list()
country_lists.append(gdp_country_list)
palma_country_list = palma_processed.index.to_list()
country_lists.append(palma_country_list)
oecd_country_list = oecd_processed.index.to_list()
country_lists.append(oecd_country_list)
democracy_country_list = democracy_processed.index.to_list()
country_lists.append(democracy_country_list)

common_list = set(country_lists[0])
for lst in country_lists:
    common_list = common_list.intersection(lst)

common_list = list(common_list)

# Filter so we have the same countries across all datasets
all_renewables = all_renewables.loc[common_list]
all_nonrenewables = all_nonrenewables.loc[common_list]
all_processed = all_processed.loc[common_list]
property_processed = property_processed.loc[common_list]
rents_processed = rents_processed.loc[common_list]
gdp_processed = gdp_processed.loc[common_list]
palma_processed = palma_processed.loc[common_list]
oecd_processed = oecd_processed.loc[common_list]
democracy_processed = democracy_processed.loc[common_list]

# Scale resources by gdp
all_renewables = all_renewables.div(gdp_processed, axis = 0)
all_nonrenewables = all_nonrenewables.div(gdp_processed, axis = 0)
all_processed = all_processed.div(gdp_processed, axis = 0)

## Creating a model
### Predicting renewables

In [565]:
X_renew = pd.concat([all_renewables["renew_change"], property_processed, rents_processed, palma_processed, democracy_processed], axis = 1)

X_renew_normal =(X_renew-X_renew.min())/(X_renew.max()-X_renew.min())
labels_renew = X_renew_normal.keys()

#### PCA Analysis

In [566]:
from sklearn.decomposition import PCA
pca = PCA(n_components=4)
pca.fit(X_renew_normal)
components = pca.components_

principalDF = pd.DataFrame(data = components, columns = labels_renew)
principalDF

,renew_change,Property rights score,Resource rents,Palma,Electoral democracy index
0,0.052939,-0.500064,0.231569,0.040000,-0.831811
1,-0.087465,-0.653660,-0.008819,0.626666,0.415078
2,-0.207070,0.113221,0.954091,-0.018891,0.183458
3,0.372945,0.511218,0.078383,0.736331,-0.226367


#### Linear Regression

In [573]:
from sklearn import linear_model
from sklearn import metrics

y = X_renew_normal[[labels_renew[0]]]
X = X_renew_normal[labels_renew[1:]]

lr_model = linear_model.LinearRegression()
lr_model.fit(X, y)
lr_model.coef_
renew_coefs = pd.DataFrame(lr_model.coef_, columns = labels[1:], index = ["Change in renewables"])
display(renew_coefs)
renew_pred = lr_model.predict(X)
r2_renew = metrics.r2_score(y, renew_pred)
print(r2_renew)

,Property rights score,Resource rents,Palma,Electoral democracy index
Change in renewables,0.028595,-0.077362,0.028645,-0.094573


0.04228436603791064


### Predicting Non-renewables

In [574]:
X_nonrenew = pd.concat([all_nonrenewables["nonrenew_change"], property_processed, rents_processed, palma_processed, democracy_processed], axis = 1)

X_nonrenew_normal =(X_nonrenew-X_nonrenew.min())/(X_nonrenew.max()-X_nonrenew.min())
labels_nonrenew = X_nonrenew_normal.keys()

#### PCA Analysis

In [575]:
from sklearn.decomposition import PCA
pca = PCA(n_components=4)
pca.fit(X_nonrenew_normal)
components = pca.components_

principalDF = pd.DataFrame(data = components, columns = labels_nonrenew)
principalDF

,nonrenew_change,Property rights score,Resource rents,Palma,Electoral democracy index
0,0.064906,-0.498836,0.231258,0.039862,-0.831794
1,-0.240606,-0.634891,0.150293,0.574811,0.431308
2,-0.164357,0.225373,0.942953,-0.146811,0.107143
3,0.507268,0.409473,0.120716,0.735954,-0.137151


In [577]:
from sklearn import linear_model

y = X_nonrenew_normal[[labels_nonrenew[0]]]
X = X_nonrenew_normal[labels_nonrenew[1:]]

lr_model = linear_model.LinearRegression()
lr_model.fit(X, y)
lr_model.coef_
nonrenew_coefs = pd.DataFrame(lr_model.coef_, columns = labels[1:], index = ["Change in nonrenewables"])
display(nonrenew_coefs)
nonrenew_pred = lr_model.predict(X)
r2_nonrenew = metrics.r2_score(y, nonrenew_pred)
print(r2_nonrenew)

,Property rights score,Resource rents,Palma,Electoral democracy index
Change in nonrenewables,0.090705,-0.070644,0.039512,-0.142085


0.0710283094555848


## Visualizing Results

In [571]:
results = pd.concat([renew_coefs, nonrenew_coefs], axis = 0)
results

,Property rights score,Resource rents,Palma,Electoral democracy index
Change in renewables,0.028595,-0.077362,0.028645,-0.094573
Change in nonrenewables,0.090705,-0.070644,0.039512,-0.142085
